In [5]:
import os
import json

In [10]:
bdir = "./jobs/"
cnt = [ json.loads(open(bdir+x, 'r').read()) for x in os.listdir(bdir)]

In [14]:
print(json.dumps(cnt[0], indent=2, ensure_ascii=False))

{
  "extra_hrs": "依工作需要", 
  "doom": "不提供住宿", 
  "pay_type": "依學經歷、證照核薪", 
  "job_id": "1760973", 
  "off_days": "週休二日", 
  "food": "", 
  "ac_deg": "不拘", 
  "job_type": "職業大貨車司機", 
  "work_exp": "不拘", 
  "hire_dur": "不定期契約", 
  "hire_num": "3", 
  "com_unicode": "88680903", 
  "work_loc": "工作地非屬專區", 
  "job_desc": "徵:職業大貨車司機(自備貨車10.5噸、15噸、17噸)，工作內容:送貨與隨車助手，意者親洽或電洽。", 
  "job_req": "全職\r\n                            \r\n                            日班：自08時00分至17時00分", 
  "ac_sch": "無", 
  "com_addr": "高雄市永安區永工五路６號", 
  "ac_dep": "不拘", 
  "job_title": "職業大貨車司機(自備貨車)", 
  "com_name": "鶴壽瓦楞紙器股份有限公司永安廠"
}


In [19]:
import collections

In [55]:
pay_types = [ x['pay_type'] for x in cnt]
pay_types_c = collections.Counter([ x['pay_type'] for x in cnt])

In [56]:
import re

In [57]:
suma = 0
for (x,y) in pay_types_c.most_common():
    if re.search(u"月薪", x):
#         print x.replace(u"月薪", "").strip(), y
        suma += y
print suma

6767


In [44]:
job_desc = [ list(jieba.cut(x['job_desc'])) for x in cnt]

In [54]:
print (pay_types[10])

0


In [68]:
def pay_class(pay):
    if pay>80000:
        return "H"
    elif pay > 35000:
        return "M"
    else:
        return "L"

In [69]:
wanted = []
for (pay, desc) in zip(pay_types, job_desc):
    if re.search(u"月薪", pay):
        t_pay = pay.replace(u"月薪", '').strip()
        (up, down) = t_pay.split("-")
        
        wanted.append(( pay_class((int(up) + int(down))/2) , desc))

In [70]:
print len(wanted)

6767


In [71]:
wanted[12]

('M',
 [u'\u63d0\u4f9b',
  u'\u7f8e\u5bb9',
  u'\u7f8e\u9ad4',
  u'\u3001',
  u'\u82b3\u9999',
  u'\u7642\u7a0b',
  u'\u3001',
  u'\u7cbe\u6cb9',
  u'\u6309\u6469',
  u'\u8212\u58d3',
  u'\u53ca',
  u'\u76f8\u95dc',
  u'\u670d\u52d9',
  u'\u4e4b',
  u'\u5de5\u4f5c\u8005',
  u'\u3002'])

In [72]:
from sklearn.cluster import KMeans

In [73]:
kmeans = KMeans(n_clusters=2, random_state=0)

In [74]:
from sklearn.feature_extraction.text import TfidfVectorizer


In [154]:
tfidf = TfidfVectorizer(lowercase=False, sublinear_tf = True, max_df = 0.85, 
                        analyzer=lambda x: x, token_pattern=None) 

In [155]:
ans = tfidf.fit_transform([desc for (mclass, desc) in wanted])

In [156]:
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)
 
def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    """get the feature names and tf-idf score of top n items"""
    
    #use only topn items from vector
    sorted_items = sorted_items[:topn]
 

    score_vals = []
    feature_vals = []
    
    # word index and corresponding tf-idf score
    for idx, score in sorted_items:
        
        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])
 
    return zip(feature_vals, score_vals)

In [159]:
feature_names=tfidf.get_feature_names()

#sort the tf-idf vectors by descending order of scores
sorted_items=sort_coo(ans.tocoo())

#extract only the top n; n here is 10
keywords=extract_topn_from_vector(feature_names,sorted_items,10)
 
# now print the results
print("\n===Keywords===")
for (w, s) in keywords:
    print("%s=%.2f"%(w, s))


===Keywords===
需輪班=1.00
運輸=1.00
送貨=1.00
送貨=1.00
送貨=1.00
研磨=1.00
白鐵學徒=1.00
焊接=1.00
清潔維護=1.00
木作=1.00


In [176]:
kmeans = KMeans(n_clusters=10, random_state=0).fit(ans)

In [195]:
aaa = {}
avg_pay = {}
idx=0
for (mdesc, label) in zip(ans, kmeans.labels_):
    if not label in aaa.keys():
        aaa[label] = []
        avg_pay[label] = []
    
    if len(aaa[label]) < 10:
        aaa[label].append("".join([ feature_names[x] for x in mdesc.tocoo().col]))
        avg_pay[label].append(wanted[idx][0])
        
    idx += 1

In [196]:
for lbl in aaa.keys():
    print "="*5, lbl, "="*5
    print "-%s"%"\n-".join(aaa[lbl])
    print avg_pay[lbl]

===== 0 =====
-、，面試。:徵時間配合需意者請能程式寫獨立工程車床操作約加班CNC師能鑽孔電洽
-、，/或負責流程產品意者請先電洽以上生產運作親洽操作機器符合標準檢視高中職畢報表檢查作業員生產線事物程序並協助填寫瑕疵寄履歷
--、，有與。的:徵負責交辦等之品完成產製作業設備生產運作造包裝操作機器出貨事宜主管機台正常目標檢驗報表作業員生產線程序填寫並維持進行現場組裝以檢視距離關生產產品
-,學徒設備操作機器
-1.3機械從事零件810傳統車床加工組立2.6尺尺車床
- 工作內容、1及。:.23能力產品等程式製作獨立作業車床操作機台CNC修改校刀校車車削件量測
-工作、，。產品組裝從事等零件設備生產包裝操作機器生產線程序根據事務運作
-處理、()1/及:.23月溝通（需人5日機個）高設備操作控制基本機台晶圓廠無塵室測機異常半導體精密度物四班二輪夜輪調
-操作機台磨床
-工作，。的負責機各種進行操作CNC加工物半熟手車削
['L', 'L', 'L', 'L', 'L', 'M', 'L', 'L', 'L', 'L']
===== 1 =====
-- 工作內容：跟車、幫忙運送等等性質(此職缺需搬重物,歡迎可者應徵，願向高薪挑戰)有EMT1初級救護技術員優先考慮*提供住宿★請以電話聯絡陳先生/0986119000聯絡者優先安排面試謝謝！徵求條件本公司救護員專業證照培訓考試及複訓無證照只要有愛心熱心興趣與。受訓課程對救護業或相關替代役消防背景從事義消方面的您主動加入行列※叫車專線:0800333119徵才000
--處理 工作內容：,有聯絡先生條件公司:.2其他人員時間900休假客服◆名稱分類需產品服務需求人數地點新北市三重區~制度薪資範圍方式刻苦耐勞騰達科技職務美編職務名稱電腦美工光明路59號1F說明修圖排版上架文書所人才MSofficePhotoShopIllustrator名60938685363鄭mailtoa3175sky@gmailcom補充須周休勞／健保面議
-,.冷凍空壓機械維修機械組裝電機馬達乾燥機過濾器
-工作：()1/相關0800.時間配合休假名稱經驗日班~依須；進度具有職位技工配管工程真空管體1700偶休假日出勤真空年以上者佳及會畫管路圖
-處理工作內容：可有/。:徵時間假日強配合客服經驗者都休商品下班行銷網頁週休二日很佳上班職者網拍